### DeBERTa センチメント分析

#### Google Driveマウント & データフォルダ設定 & ライブラリ設定

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install transformers datasets pandas torch

#### モデルとトークナイザーの読み込み

In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F

# モデルとトークナイザーのロード
model_name = "./DeBERTa/best_model_f1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

In [ ]:
import re

# 【】の文字列を削除する関数
def remove_prefix(text):
    # 正規表現パターン: 「任意の一文字」と「【】で囲まれた文字列」
    pattern = r'.【.*?】'
    # 置換: マッチした部分を空文字列に置換
    result = re.sub(pattern, '', text)
    return result

In [ ]:
# モデルを評価モードに設定
model.eval()

# デバイスの設定 (GPUが利用可能な場合)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
# チャンクごとにデータを処理する関数
def process_chunk(chunk, text_column_index):
    texts = chunk.iloc[:, text_column_index].tolist()
    texts = [remove_prefix(text) for text in texts]
    print(texts)

    # データの前処理
    def preprocess_function(texts):
        return tokenizer(texts, padding='max_length', truncation=True, max_length=128, return_tensors="pt")

    inputs = preprocess_function(texts)

    with torch.no_grad():
        inputs = {key: val.to(device) for key, val in inputs.items()}
        outputs = model(**inputs)
        probs = F.softmax(outputs.logits, dim=-1)
        positive_probs = probs[:, 1].cpu().numpy()  # クラス1（ポジティブ）の確率
        negative_probs = probs[:, 0].cpu().numpy()  # クラス0（ネガティブ）の確率

    select_columns = [0, 1, 2, 3, 4, 6, 7]
    result_chunk = chunk.iloc[:, select_columns].copy()  # 1～5列目，7,8列目をコピー
    result_chunk['positive_probability'] = positive_probs
    result_chunk['negative_probability'] = negative_probs

    return result_chunk

In [ ]:
import os
# 一時ファイルディレクトリの作成
temp_dir = "./temp_chunks"
os.makedirs(temp_dir, exist_ok=True)

In [ ]:
# CSVファイルの読み込みと処理
input_csv = './MDA_DataSet_2014_2022_TSE1.csv'  # 入力ファイルパス
output_csv = './MDA_DataSet_2014_2022_TSE1_deberta.csv'  # 出力ファイルパス
chunk_size = 50  # チャンクサイズ

In [ ]:
# チャンクごとに処理して一時ファイルに保存
chunk_number = 0
for chunk in pd.read_csv(input_csv, chunksize=chunk_size):
    processed_chunk = process_chunk(chunk, text_column_index=5)  # テキスト列のインデックスを6列目に指定（0から数えるので5）
    temp_chunk_file = os.path.join(temp_dir, f"chunk_{chunk_number}.csv")
    processed_chunk.to_csv(temp_chunk_file, index=False)
    chunk_number += 1

# 一時ファイルを結合して最終結果ファイルを作成
header_written = False
with open(output_csv, 'w', encoding='utf-8') as fout:
    for i in range(chunk_number):
        temp_chunk_file = os.path.join(temp_dir, f"chunk_{i}.csv")
        with open(temp_chunk_file, 'r', encoding='utf-8') as fin:
            if not header_written:
                # ヘッダーを書き込む
                fout.write(fin.read())
                header_written = True
            else:
                # ヘッダーをスキップして書き込む
                next(fin)
                fout.write(fin.read())

# 一時ファイルの削除
for i in range(chunk_number):
    temp_chunk_file = os.path.join(temp_dir, f"chunk_{i}.csv")
    os.remove(temp_chunk_file)
os.rmdir(temp_dir)

print("処理が完了しました。")